<a href="https://colab.research.google.com/github/Alokik-29/Assign.1/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Cell 1: Install all required dependencies
!pip install -q langchain langchain-community chromadb sentence-transformers

In [10]:
# Cell 2: Install and setup Ollama in Colab
# This installs Ollama and pulls the Mistral model
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server in background
import subprocess
import time

# Start Ollama server
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("Waiting for Ollama server to start...")
time.sleep(5)

# Pull Mistral model
!ollama pull mistral

print("Ollama setup complete!")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Waiting for Ollama server to start...

Ollama setup complete!


In [11]:
# Cell 3: Import all necessary libraries
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("All imports successful!")

All imports successful!


In [12]:
# Cell 4: Load the speech text and split it into chunks
# Load the text file from Colab storage
loader = TextLoader("speech.txt", encoding='utf-8')
documents = loader.load()

print(f"Loaded {len(documents)} document(s)")
print(f"Document length: {len(documents[0].page_content)} characters")

# Split the text into manageable chunks
# chunk_size: Number of characters per chunk
# chunk_overlap: Number of overlapping characters between chunks for context continuity
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separator="\n"
)

# Create chunks from the loaded documents
chunks = text_splitter.split_documents(documents)

print(f"\nSplit into {len(chunks)} chunks")
print(f"\nFirst chunk preview:\n{chunks[0].page_content[:200]}...")

Loaded 1 document(s)
Document length: 757 characters

Split into 2 chunks

First chunk preview:
The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to succeed if you allow the shastras to continue to be 
held as sacred and infallible? You must take a stand...


In [13]:
# Cell 5: Create embeddings and store in ChromaDB
# Initialize HuggingFace embeddings with the specified model
# This model converts text into numerical vectors for semantic search
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("Creating embeddings and storing in ChromaDB...")
print("This may take a moment on first run as the model downloads...")

# Create ChromaDB vector store from the document chunks
# persist_directory: Local folder where the vector database is stored
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"\n✓ Vector store created successfully!")
print(f"✓ Stored {vectorstore._collection.count()} vectors in ChromaDB")

Creating embeddings and storing in ChromaDB...
This may take a moment on first run as the model downloads...

✓ Vector store created successfully!
✓ Stored 2 vectors in ChromaDB


In [14]:
# Cell 6: Initialize Ollama LLM and create the RAG chain
# Initialize Ollama with Mistral model
llm = Ollama(
    model="mistral",
    temperature=0.2  # Lower temperature for more focused, factual responses
)

print("Testing Ollama connection...")
test_response = llm.invoke("Hello")
print(f"✓ Ollama is working! Test response: {test_response[:50]}...\n")

# Create a custom prompt template for better responses
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer based on the context provided, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Answer: """

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create the RetrievalQA chain
# This chain retrieves relevant chunks and generates answers using the LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" means all retrieved docs are stuffed into the prompt
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 3 most relevant chunks
    return_source_documents=True,  # Return the source chunks used for answering
    chain_type_kwargs={"prompt": PROMPT}
)

print("✓ RAG Chain created successfully!")
print("✓ System ready to answer questions!")

Testing Ollama connection...


/tmp/ipython-input-643226795.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


✓ Ollama is working! Test response:  Hello! How can I assist you today?

I'm here to h...

✓ RAG Chain created successfully!
✓ System ready to answer questions!


In [18]:
# Cell 7: Define function to ask questions
def ask_question(question):
    """
    Ask a question based on the Dr. Ambedkar speech content.

    Args:
        question (str): The question to ask

    Returns:
        None: Prints the answer and source documents
    """
    print(f"\n{'='*80}")
    print(f"QUESTION: {question}")
    print(f"{'='*80}\n")

    # Get response from the RAG chain
    response = qa_chain.invoke({"query": question})

    # Print the answer
    print(f"ANSWER:\n{response['result']}\n")

    # Print source documents used
    print(f"{'-'*80}")
    print(f"SOURCE DOCUMENTS USED:")
    print(f"{'-'*80}")
    for i, doc in enumerate(response['source_documents'], 1):
        print(f"\n[Source {i}]")
        print(doc.page_content[:300] + "..." if len(doc.page_content) > 300 else doc.page_content)

    print(f"\n{'='*80}\n")

print("✓ Question answering function ready!")

✓ Question answering function ready!
